In [1]:
import pandas as pd
import requests
from datetime import timedelta, datetime
from io import StringIO
from jinja2 import Environment, FileSystemLoader

In [2]:
N_DAYS_AGO = 7
today = datetime.now()    
n_days_ago = today - timedelta(days=N_DAYS_AGO)
today = today.strftime("%Y-%m-%d")
prior = n_days_ago.strftime("%Y-%m-%d")

In [3]:
df_old = pd.read_csv(f"./forecast_data/usaid-business-forecast-{prior}.csv")

In [4]:
with requests.Session() as s:
    download = s.get('https://www.usaid.gov/business-forecast/export/csv?page&_format=csv')
    decoded = download.content.decode('utf-8')
    df_new = pd.read_csv(StringIO(decoded), sep=',')

In [5]:
df_new.to_csv(f"./forecast_data/usaid-business-forecast-{today}.csv")

In [6]:
def fh_only (df):
    new_df = df[(df["Operating Unit"].isin([
        'Bangladesh',
        'Bolivia',
        'Burundi',
        'Cambodia',
        'Democratic Republic of the Congo',
        'Dominican Republic',
        'Ethiopia',
        'Guatemala',
        'Haiti',
        'Indonesia',
        'Kenya',
        'Mozambique',
        'Nicaragua',
        'Peru',
        'Philippines',
        'Rwanda',
        'South Sudan',
        'Uganda',
        'Bureau for Conflict Prevention and Stabilization (CPS)',
        "Regional Development Mission for Asia",
        "Bureau for Humanitarian Assistance (BHA)",
        "Innovation, Technology, and Research Hub (DDI/ITR)",
        "Private-Sector Engagement Hub (DDI/PSE)",
        "Bureau for Resilience and Food Security (RFS)",
        "Local, Faith, and Transformative Partnerships Hub (DDI/LFT)",
        "Center for Environment, Energy, and Infrastructure (DDI/EEI)",
        "Center for Democracy, Human Rights, and Governance (DDI/DRG)",
        "Office of Population & Reproductive Health (GH/PRH)",
        "Office of Maternal and Child Health and Nutrition (GH/MCHN)",
        "Gender Equality and Women's Empowerment Hub (DDI/GEWE)",
        "Center for Accelerating Innovation and Impact (GH/CAII)",
        "Inclusive Development Hub (DDI/ID)",
        "Office of Acquisition and Assistance (M/OAA)"
    ]))]
    return new_df

In [7]:
df_old = fh_only(df_old)
df_new = fh_only(df_new)

In [8]:
identifying_column = 'A&A Plan Id'

In [9]:
old_list = df_old[identifying_column].values.tolist()
new_list = df_new[identifying_column].values.tolist()

In [10]:
removed = list(set(old_list) - set(new_list))
added = list(set(new_list) - set(old_list))

In [11]:
removed_df = df_old[df_old[identifying_column].isin(removed)]
df_old_final = df_old[~df_old[identifying_column].isin(removed)]


added_df = df_new[df_new[identifying_column].isin(added)]
df_new_final = df_new[~df_new[identifying_column].isin(added)]


df_new_final = df_new_final.sort_values(by=[identifying_column])
df_old_final = df_old_final.sort_values(by=[identifying_column])


df_new_final = df_new_final.reset_index(drop=True)
df_old_final = df_old_final.reset_index(drop=True)

comparison_df = df_new_final.compare(df_old_final)

In [12]:
iteration_keys = []
for i in comparison_df.index:
    iteration_keys.append(
        {
            'title': df_new_final.iloc[i]['Award Title'],
            'url': df_new_final.iloc[i]['URL'],
            'changes' : {},
            'ou': df_new_final.iloc[i]['Operating Unit']
        }
    )

In [13]:
comparison_df = comparison_df.reset_index(drop=True)

In [14]:
index = 0
while index < len(comparison_df):
    for i, v in zip(comparison_df.iloc[index].index, comparison_df.iloc[index]):
        if pd.isnull(comparison_df.iloc[index][i[0]]['self']) and pd.isnull(comparison_df.iloc[index][i[0]]['other']):
            continue
        else:
            if i[1] == 'self':
                iteration_keys[index]['changes'][i[0]] = v
            else:
                iteration_keys[index]['changes'][i[0]] = [v , iteration_keys[index]['changes'][i[0]]]
    index += 1

In [15]:
environment = Environment(loader=FileSystemLoader("./"))
template = environment.get_template("jinja_template.html")

In [16]:
def create_objs(df):
    titles = list(df['Award Title'])
    urls = list(df['URL'])
    ou = list(df['Operating Unit'])

    base_list = []

    index = 0
    while index < len(titles):
        base_list.append(
            {
                'title': titles[index],
                'url': urls[index],
                'ou': ou[index]
            }
        )

        index += 1

    return base_list

In [17]:
content = template.render(
        removed=create_objs(removed_df),
        added=create_objs(added_df),
        changed = iteration_keys
    )

with open(f"usaid_bf_weekly_report_{datetime.now().strftime('%Y%m%d')}.html", mode="w") as message:
    message.write(content)